In [ ]:
import copy
import os
import numpy as np
import sys
import types
from scipy import signal
from scipy.interpolate import CubicSpline
from scipy.stats import norm  # for u(t) as gaussians

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec

In [ ]:
%matplotlib ipympl
#%matplotlib inline

In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# Notebook setup (path trick) and local import

In [ ]:
SRC_ROOT = os.path.dirname(os.path.abspath(''))
print('appending to path SRC_ROOT...', SRC_ROOT)
sys.path.append(SRC_ROOT)

PACKAGE_ROOT = os.path.dirname(os.path.dirname(os.path.abspath('')))
print('appending to path PACKAGE_ROOT...', PACKAGE_ROOT)
sys.path.append(PACKAGE_ROOT)

NB_OUTPUT = SRC_ROOT + os.sep + 'output'

if not os.path.exists(NB_OUTPUT):
    os.makedirs(NB_OUTPUT)

# Plotting functions

In [ ]:
color_input = '#00AEEF'
#color_memory = '#DEB87A'  
color_memory = '#C1A16B'  # dark: #DEB87A || darker: #D0AD73 ||| darkest: #C1A16B
color_response = '#662D91'

linewidth = 1.5
linestyle = '-'

line_kwargs_input = dict(color=color_input, linewidth=linewidth, linestyle=linestyle)
line_kwargs_memory = dict(color=color_memory, linewidth=linewidth, linestyle=linestyle)
#line_kwargs_memory_sigma = dict(color=color_memory, linewidth=linewidth, linestyle='dotted')
line_kwargs_memory_sigma = dict(color=color_memory, linewidth=linewidth, linestyle=linestyle)
line_kwargs_response = dict(color=color_response, linewidth=linewidth, linestyle=linestyle)

In [ ]:
#%matplotlib widget
%matplotlib ipympl

## 0. Construct some signals u(t)

In [ ]:
def signal_rectangle(n_pulse, n_rest, n_cycles, period_S1=1.0, times=None, duty=0.01):
    
    def ubase_rect(t, t_mid, duty, period_S1):
        scale_for_unit_area = 1 / (duty * period_S1)
        val_at_t = scale_for_unit_area * (
            np.heaviside(t - (t_mid - duty * period_S1), 0) - 
            np.heaviside(t - t_mid, 0))
        return val_at_t
    
    L_cycle = n_pulse + n_rest
    
    if times is None:
        dt = 0.001
        tmax = L_cycle * n_cycles * period_S1
        times = np.arange(-2, tmax + dt, dt)
    else:
        dt = times[1] - times[0]
        
    tmid_list = [period_S1*(n + L_cycle * k) for k in range(n_cycles) for n in range(1, n_pulse+1)]
    
    # choose pulse_area
    pulse_area = 1.0
    
    # build u: wave of rectangles
    assert duty * period_S1 > 5 * dt  # want it to be sampled nicely
    u_rect = pulse_area  * np.sum([ubase_rect(times, t_mid, duty, period_S1) for t_mid in tmid_list], axis=0)   
    
    return times, u_rect


def signal_gaussian(n_pulse, n_rest, n_cycles, period_S1=1.0, times=None, sigma=0.05):
    
    def ubase_gaussian(t, t_mid, sigma):
        # direct calc has exp overflow
        '''sigma_sqr = sigma ** 2 
        prefactor = 1 / np.sqrt(2 * np.pi * sigma_sqr)
        expval = (t - t_mid) ** 2 / (2 * sigma_sqr)
        val_at_t = prefactor * np.exp(expval)'''
        val_at_t = norm.pdf(t, t_mid, sigma)
        return val_at_t
    
    L_cycle = n_pulse + n_rest
    
    if times is None:
        dt = 0.001
        tmax = L_cycle * n_cycles * period_S1
        times = np.arange(-2, tmax + dt, dt)
    else:
        dt = times[1] - times[0]
    
    tmid_list = [period_S1*(n + L_cycle * k) for k in range(n_cycles) for n in range(1, n_pulse+1)]
    
    # choose pulse_area
    pulse_area = 1.0
    
    # build u: wave of rectangles
    u_gaussian = pulse_area * np.sum([ubase_gaussian(times, t_mid, sigma) for t_mid in tmid_list], axis=0)
    
    return times, u_gaussian

In [ ]:
def W_of_t_manual(t, u, rate_decay, rate_grow):
    "As opposed to 1/dt * prate * np.convolve(u, exp_of_t, 'same')"    
    
    def do_integral(t_specific, t_idx):

        # could drop the heaviside factor if we can restrict limits of integration?
        #integrand = u * np.exp(alpha * (t - t_specific)) *  np.heaviside(t_specific - t, 1)
        
        ur = u[:t_idx]
        tr = t[:t_idx]
        integrand = ur * np.exp(rate_decay * (tr - t_specific))
        
        return np.trapz(integrand, dx=dt)
    
    W_of_t = np.zeros_like(t)
    print('Working on W_of_t_manual integrals...')
    for idx, tau in enumerate(t):
        W_of_t[idx] = do_integral(tau, idx)
    
    W_of_t = rate_grow * W_of_t
    
    return W_of_t

def W_of_t_convolve(t, u, rate_decay, rate_grow, nmult=5):
    "Simple (fast) Alternative to ODE or direct integration: use convolution"  
    dt = t[1] - t[0]
    nn = len(t)
    
    # size of window depends on alpha, slower (smaller) alpha means longer window
    tsample_exp_window = np.arange(0, (nmult * nn) * dt + dt, dt)
    exp_of_t_window = np.exp(-rate_decay * tsample_exp_window)
    assert len(tsample_exp_window) > nn

    W_of_t = dt * rate_grow * signal.convolve(u, exp_of_t_window, 'full')  
    W_of_t = W_of_t[0:nn]  # truncate to early values matching length of u
    return W_of_t


def filter_pointwise_expsimple(t, u, rate_decay):
    return np.exp(-rate_decay * t)


def filter_pointwise_expmemory_tanh(t, u, rate_decay, rate_grow, plot=False):

    #x_of_t = W_of_t_manual(t, u, alpha)
    x_of_t = W_of_t_convolve(t, u, rate_decay, rate_grow)
    
    print('u', len(u))
    print('x_of_t', len(x_of_t))
    
    r_of_t_tanh = 1 - np.tanh(x_of_t)
    y_of_t_tanh = u * r_of_t_tanh
    
    if plot:
        plt.figure()
        plt.plot(t, u, linewidth=0.25, label=r'$u(t)$')
        plt.plot(t, x_of_t, label=r'$x(t)$: weighted sum')
        plt.plot(t, r_of_t_tanh, label=r'$r(t) = 1 - \mathrm{tanh}(x)$')
        plt.plot(t, y_of_t_tanh, label=r'$y(t) = u(t) (1 - \mathrm{tanh}(x))$')
        plt.grid(alpha=0.5)
        plt.title('plot inside filter_pointwise_expmemory')
        plt.xlabel('t')
        plt.legend()   

    return r_of_t_tanh


def filter_pointwise_expmemory_hill(t, u, rate_decay, rate_grow, N=2, plot=False):
    
    x_of_t = W_of_t_convolve(t, u, rate_decay, rate_grow)
    
    print('u', len(u))
    print('W_of_t', len(x_of_t))
    
    r_of_t = 1 / (1 + x_of_t ** N)
    
    print(len(t))
    print(len(u))
    print(len(x_of_t))
    print(len(r_of_t))
    
    y_of_t = u * r_of_t
    
    if plot:
        plt.figure()
        plt.plot(t, u, linewidth=0.25, label=r'$u(t)$')
        plt.plot(t, x_of_t, label=r'$x(t)$: weighted sum')
        plt.plot(t, r_of_t, label=r'$r(t) = 1 / (1 + x^N)$')
        plt.plot(t, y_of_t, label=r'$y(t) = u(t) / (1 + x^N)$')
        plt.grid(alpha=0.5)
        plt.title('plot inside filter_pointwise_expmemory')
        plt.xlabel('t')
        plt.legend()   
    
    return r_of_t

In [ ]:
period_S1 = 1.0 # period between consecutive pulses
n_pulse = 10
n_rest = 1
L_cycle = n_pulse + n_rest
n_cycles = 1  # 3
tmid_list = [period_S1*(n + L_cycle * k) for k in range(n_cycles) for n in range(1, n_pulse+1)]

# prep t to sample signals from
dt = 0.001
#tmax = L_cycle * period_S1 * 4
tmax = L_cycle * n_cycles * period_S1
t = np.arange(-0.01, tmax + dt, dt)

# choose pulse_area
pulse_area = 1.0

# build u: heaviside step
u_step = np.heaviside(t, 0)

# build u: heaviside staircasec
u_stairs = np.sum([np.heaviside(t - t_mid, 0) for t_mid in tmid_list], axis=0)   

# build u: wave of gaussians
_, u_gaussian = signal_gaussian(n_pulse, n_rest, n_cycles, period_S1=period_S1, times=t, sigma=0.05)
_, u_gaussian_wide = signal_gaussian(n_pulse, n_rest, n_cycles, period_S1=period_S1, times=t, sigma=0.1)

# build u: wave of rectangles
_, u_rect = signal_rectangle(n_pulse, n_rest, n_cycles, period_S1=period_S1, times=t, duty=0.01)    
_, u_rect_wide = signal_rectangle(n_pulse, n_rest, n_cycles, period_S1=period_S1, times=t, duty=0.25)

plt.figure()
ax = plt.gca()
ax.plot(t, u_step, label=r'$u(t)$ step')
ax.plot(t, u_stairs, label=r'$u(t)$ staircase')
ax.plot(t, u_gaussian_wide, label=r'$u(t)$ gaussian pulses')
ax.plot(t, u_rect_wide, label=r'$u(t)$ rectangle pulses')
ax.axhline(0, linestyle='-', alpha=0.2, color='k')
ax.grid(alpha=0.5)
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'signal')
ax.legend()
#plt.xlim(0.9, 1.1)
plt.title(r'Example input signals $u(t)$')
plt.show()

### Alt plots

In [ ]:
def MOD_plot_uxry_stack(t, u, rate_decay, rate_grow, title, fpath):
    """
    Args:
        fmod: 'rectangle', 'gaussian'
    """
    
    x_of_t = W_of_t_convolve(t, u, rate_decay, rate_grow)
    r_of_t = filter_pointwise_expmemory_hill(t, u, rate_decay, rate_grow, N=2, plot=False)
    y_of_t = u * r_of_t 

    # plot params
    grid_alpha = 0.6

    plt.close(); 
    fig, axarr = plt.subplots(4, 1, sharex=True, squeeze=False, figsize=(7, 5))

    axarr[0, 0].set_title(title)
    axarr[0,  0].set_ylabel(r'input $u(t)$')
    axarr[1,  0].set_ylabel(r'memory $x(t)$')
    axarr[2,  0].set_ylabel(r'filter $r(t)$')
    axarr[3,  0].set_ylabel(r'output $y(t)$')
    axarr[-1, 0].set_xlabel(r'$t$')

    #axarr[0,0].set_title(r'$foo 1$')
    axarr[0,0].plot(t, u, label=r'$u(t)$')

    #axarr[1,0].set_title(r'$foo 2$')
    axarr[1,0].plot(t, x_of_t, label=r"$x(t)=\int_{0}^t \,\beta e^{-\alpha (t-t')} u(t') dt'$")
    axarr[1,0].axhline(1, linestyle='--', alpha=0.5)
    axarr[1,0].set_ylim(-0.05, np.max(x_of_t)*1.05)

    #axarr[2,0].set_title(r'$foo 3$')
    axarr[2,0].plot(t, r_of_t, label=r'$r(t)=\sigma(x(t))$')
    axarr[2,0].axhline(0.5, linestyle='--', alpha=0.5)
    axarr[2,0].set_ylim(-0.05, 1.05)

    #axarr[3,0].set_title(r'foo 4')
    axarr[3,0].plot(t, y_of_t, label=r'$y(t)=u(t) r(t)$')    
    axarr[3,0].axhline(0.5 * np.max(u), linestyle='--', alpha=0.5)
    
    for idx in range(0, 4):
        if idx > 0:
            axarr[idx,0].plot(t, u, '-k', alpha=0.1)
        axarr[idx,0].grid(alpha=grid_alpha)
        axarr[idx,0].legend()
    
    plt.savefig(fpath)
    plt.show()
    return axarr

In [ ]:
u = u_gaussian_wide
fmod = 'rectangle'
rate_decay = 0.1
rate_grow = 0.5

title = r'%s $u(t)$ with nonlinear filter: $\alpha=%.2f$, $\beta=%.2f$' % (fmod, rate_decay, rate_grow)
fpath = NB_OUTPUT + os.sep + 'mod_uxry_stack_%s.pdf' % fmod

_ = MOD_plot_uxry_stack(t, u, rate_decay, rate_grow, title, fpath)

# Plot for Fig. 3 and ppt

In [ ]:
flag_ylabels = False
flag_legends = False

flag_grid = False
flag_right_and_top_ax = False
flag_axhlines = False

# plot params
grid_alpha = 0.6

In [ ]:
def plot_uxry_stack(t, u, rate_decay, rate_grow, title, fpath):
    """
    Args:
        fmod: 'rectangle', 'gaussian'
    """
    
    x_of_t = W_of_t_convolve(t, u, rate_decay, rate_grow)
    r_of_t = filter_pointwise_expmemory_hill(t, u, rate_decay, rate_grow, N=2, plot=False)
    y_of_t = u * r_of_t 
    amp = np.max(u)

    # plot params
    grid_alpha = 0.6

    plt.close(); 
    fig, axarr = plt.subplots(4, 1, sharex=True, squeeze=False, figsize=(7, 5))

    axarr[0, 0].set_title(title)
    if flag_ylabels:
        axarr[0,  0].set_ylabel(r'input $u(t)$')
        axarr[1,  0].set_ylabel(r'memory $x(t)$')
        axarr[2,  0].set_ylabel(r'filter $r(t)$')
        axarr[3,  0].set_ylabel(r'output $y(t)$')
    axarr[-1, 0].set_xlabel(r'$t$')

    #axarr[0,0].set_title(r'$foo 1$')
    axarr[0,0].plot(t, u, zorder=10, label=r'$u(t)$', **line_kwargs_input)
    axarr[0,0].set_yticks([0, amp])
    axarr[0,0].set_yticklabels(['0', '%d' % int(amp)])
    
    #axarr[1,0].set_title(r'$foo 2$')
    axarr[1,0].plot(t, x_of_t, zorder=10, label=r"$x(t)=\int_{0}^t \,\beta e^{-\alpha (t-t')} u(t') dt'$", **line_kwargs_memory)
    if flag_axhlines:
        axarr[1,0].axhline(1, linestyle='--', alpha=0.5, color='k')
    axarr[1,0].set_ylim(-0.05, np.max(x_of_t)*1.05)

    #axarr[2,0].set_title(r'$foo 3$')
    axarr[2,0].plot(t, r_of_t, zorder=10, label=r'$r(t)=\sigma(x)$', **line_kwargs_memory_sigma)
    if flag_axhlines:
        axarr[2,0].axhline(0.5, linestyle='--', alpha=0.5, color='k')
    axarr[2,0].set_ylim(-0.05, 1.05)
    axarr[2,0].set_yticks([0, 0.5, 1.0])
    axarr[2,0].set_yticklabels(['0', '', '1'])

    #axarr[3,0].set_title(r'foo 4')
    #axarr[3,0].plot(t, amp * r_of_t, zorder=10, label=r'$r(t)=\sigma(x(t))$', **line_kwargs_memory_sigma)
    axarr[3,0].plot(t, y_of_t, zorder=10, label=r'$y(t)=u(t) r(t)$', **line_kwargs_response)
    if flag_axhlines:
        axarr[3,0].axhline(0.5 * np.max(u), linestyle='--', alpha=0.5, color='k')
    
    for idx in range(0, 4):
        if idx > 0:
            height_scale = axarr[idx,0].get_ylim()[1] / np.max(u)
            axarr[idx,0].plot(t, u * height_scale, '-k', alpha=0.05, zorder=1)
        if flag_grid:
            axarr[idx,0].grid(alpha=grid_alpha)
        if not flag_right_and_top_ax:
            axarr[idx,0].spines[['right', 'top']].set_visible(False)
        
        if flag_legends:
            axarr[idx,0].legend()
        
    
    plt.xlim(-5, 60)
    plt.savefig(fpath + '.pdf')
    plt.savefig(fpath + '.svg')
    plt.show()
    return axarr

## Create plots using multiple plot fn options above 

In [ ]:
period_S1 = 1.0  # period between consecutive pulses
n_pulse = 5
n_rest = 10
L_cycle = n_pulse + n_rest
n_cycles = 3  # 3
tmid_list = [period_S1 * (n + L_cycle * k) for k in range(n_cycles) for n in range(1, n_pulse + 1)]

# prep t to sample signals from
dt = 0.001
#tmax = L_cycle * period_S1 * 4
tmax = 100 #L_cycle * n_cycles * period_S1
t = np.arange(-4, tmax + dt, dt)

# choose pulse_area
pulse_area = 1.0

_, u_rect_wide_3p = signal_rectangle(n_pulse, n_rest, n_cycles, period_S1=1.0, times=t, duty=0.2)

u = u_rect_wide
fmod = 'rectangle'
rate_decay = 0.1
rate_grow = 0.5

title = r'%s $u(t)$ with nonlinear filter: $\alpha=%.2f$, $\beta=%.2f$' % (fmod, rate_decay, rate_grow)
fpath = NB_OUTPUT + os.sep + 'mod_uxry_stack_%s' % fmod

_ = plot_uxry_stack(t, u_rect_wide_3p, rate_decay, rate_grow, title, fpath)


# Heatmap of 2D output function - consider overlaying phaseplot of trajectory
e.g. $y(t)=(1-x_1) x_2$

In [ ]:
# construct y(t) output functions, parameters, and titles
# ======================================================
def y_satlin(x1, x2):
    # x1 like memory, x2 like u(t) input
    return (1 - x1) * x2
title_satlin = r'$u (1 - x)$'

hill_N = 2
def y_hill(x1, x2):
    # x1 like memory, x2 like u(t) input
    return x2 / (1 + x1 ** hill_N)
#title_hill = r'$u / (1+x^N)$, $N=%d$' % hill_N
title_hill = r'$u / (1+x^2)$'

relu_theta = 0.0
assert relu_theta == 0.0
def y_relu(x1, x2):
    # x1 like memory, x2 like u(t) input
    relu_arg = x2 - x1 - relu_theta
    print(relu_arg.shape)
    return np.where(relu_arg>0, relu_arg, 0)
title_relu = r'$\mathrm{ReLu}(u-x)$'

fnc_title_pairs = [(y_hill, title_hill),
                   (y_satlin, title_satlin),
                   (y_relu, title_relu),
                   ]
n_fnc = len(fnc_title_pairs)

# create mesh
# ======================================================
npts = 100
x, u = np.meshgrid(np.linspace(0, 1, npts), np.linspace(0, 1, npts))

# create plot
# ======================================================
fig, axarr = plt.subplots(1, n_fnc, figsize=(9, 2.2))
#fig, axarr = plt.subplots(1, n_fnc, figsize=(5, 2.5))

for idx, pair in enumerate(fnc_title_pairs):
    
    foo_y, foo_title = pair
    ax = axarr[idx]
    
    y = foo_y(x, u)
    
    contour_color = 'k'  # 'white'
    
    cs = ax.contourf(x, u, y, levels=10, cmap='Purples_r')   # could use pcolormesh(..., shading='gouraud') instead if want smooth
    cs_contours = ax.contour(x, u, y, [0.5], linestyles='--', colors=[contour_color])
    ax.clabel(cs_contours, inline=True, fontsize=12, colors=[contour_color])  #, inline_spacing=.5)
    
    ax.set_xlabel(r'$x(t)$')
    if idx == 0:
        ax.set_ylabel(r'$u(t)$') 
    
    ax.set_title(foo_title)
    fig.colorbar(cs, ax=ax, shrink=0.9)
    
    '''
    cs2 = ax2.pcolormesh(x, y, z, shading='gouraud')
    cs2_contours = ax2.contour(x, y, z, [0.5], linestyles='--')
    ax2.set_xlabel(r'$x_1$'); ax2.set_ylabel(r'$x_2$')
    ax2.set_title("pcolormesh")
    fig.colorbar(cs2, ax=ax2, shrink=0.9)'''
    
plt.suptitle(r'Nonlinear response functions $y(t)=g(x, u)$')
plt.subplots_adjust(wspace=0.35, top=0.7)
#fig.subplots_adjust(top=0.8)

fpath = NB_OUTPUT + os.sep + 'g_of_x_u_heatmaps'
plt.savefig(fpath + '.svg')
plt.savefig(fpath + '.png')
plt.show()

In [ ]:
z = np.linspace(0, 5, 10000)
grid_alpha = 0.6
fs=12

plt.close(); 
fig, axarr = plt.subplots(1,1, sharey=True, squeeze=False, figsize=(2.2,2.7))

###axarr[0,0].set_ylabel(r'$\sigma(x)$')
axarr[0,0].set_xlabel(r'$x$', fontsize=fs+2)

axarr[0,0].set_title(r'$1/(1+x^N)$', fontsize=fs+2)
axarr[0,0].grid(alpha=grid_alpha)

axarr[0,0].plot(z, 1/(1+z ** 1), label=r'$N=1$', c='#D3D3D3')
axarr[0,0].plot(z, 1/(1+z ** 2), label=r'$N=2$', c='#899499')
axarr[0,0].plot(z, 1/(1+z ** 8), label=r'$N=8$', c='#36454F')

axarr[0,0].legend()
axarr[0,0].axhline(0.5, linestyle='--', c='k', alpha=0.5, zorder=1)

axarr[0,0].set_yticks([0, 0.5, 1.0])
axarr[0,0].set_yticklabels(['0', '', '1'], fontsize=fs)
axarr[0,0].tick_params(axis='x', labelsize=fs)


plt.tight_layout()

fpath = NB_OUTPUT + os.sep + 'hill_fn_example'
plt.savefig(fpath + '.svg')
plt.savefig(fpath + '.pdf')
plt.show()